In [6]:
# imports encryption
# from encryption import encryption
import pickle

# encryp = encryption()
file = "image/test.png"
message = "holis."

In [3]:
encryp = pickle.load(
    open("encryption.pkl", "rb")
)

In [7]:
# testing
secretKey = encryp.encrypt(message, file)
# encryp.seeImage()

In [8]:
message2 = encryp.decrypt(file, secretKey)
# encryp.seeFrame()
print(message2)

holis.


In [5]:
from encriptation_dict import create_dict

create_dict()

NameError: name 'encriptation_dict' is not defined